# Heart Rate Forecasting - Lag-Llama

In [ ]:
# Paso 1: Clonar repos e instalar dependencias
# IMPORTANTE: correr esta celda sola, luego reiniciar el runtime antes de continuar
!git clone https://github.com/AllanDBB/heartrate-forecasting.git
%cd heartrate-forecasting
!git clone https://github.com/time-series-foundation-models/lag-llama.git
!pip install -q -r lag-llama/requirements.txt
!pip install -q openpyxl scikit-learn

# Forzar restart del runtime para que numpy/pandas nuevos tomen efecto
print("✅ Instalación completa. Reiniciando runtime automáticamente...")
import os
os.kill(os.getpid(), 9)

In [ ]:
# Paso 2: Después del restart, volver al directorio correcto y descargar checkpoint
%cd /content/heartrate-forecasting

from huggingface_hub import hf_hub_download
hf_hub_download(
    repo_id='time-series-foundation-models/Lag-Llama',
    filename='lag-llama.ckpt',
    local_dir='.'
)
print('✅ Checkpoint descargado!')

In [ ]:
# Verificar GPU
import torch
print('GPU available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('Device:', torch.cuda.get_device_name(0))
    print(f'VRAM libre: {torch.cuda.mem_get_info()[0]/1e9:.2f} GB')

In [ ]:
# Cargar y preparar datos
import utils
from sklearn.model_selection import train_test_split

folder_path = 'dataset/'
dfSelectedMean = utils.loadAllFiles(folder_path)

inp = 200
out = 200

df_70, df_30 = utils.selectRandomColumns(dfSelectedMean)

pathEst_70 = 'dataset/values_deses_70.csv'
pathEst_30 = 'dataset/values_deses_30.csv'

df_scaled_70, values_deses_70 = utils.estandarizar(df_70, pathEst_70)
df_scaled_30, values_deses_30 = utils.estandarizar(df_30, pathEst_30)

X_train, y_train, ids_list_tr = utils.series_to_supervised_matrix(
    df_scaled_70.iloc[:, :-2], input_size=inp, output_size=out
)
X_tr, X_va, y_tr, y_va = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

X_test, y_test, ids_list_te = utils.series_to_supervised_matrix(
    df_scaled_30.iloc[:, :-2], input_size=inp, output_size=out
)

print('Datasets splitted and ready!')
print('Shapes:', X_tr.shape, y_tr.shape, X_va.shape, y_va.shape, X_test.shape, y_test.shape)

In [ ]:
# Cargar wrapper de Lag-Llama
import os, sys
os.chdir('/content/heartrate-forecasting')
sys.path.insert(0, '/content/heartrate-forecasting')

from wrappers.LagLlamaSupervisedWrapper import LagLlamaSupervisedWrapper

wrapper_ll = LagLlamaSupervisedWrapper(
    config_path='configs/lagllama_config.yaml',
    lag_llama_repo_path='lag-llama'
)

In [ ]:
# Smoke test con 5 muestras
y_pred_smoke = wrapper_ll.predict(X_test[:5])
print('Shape predicción:', y_pred_smoke.shape)  # esperado: (5, 200)

In [ ]:
# Evaluación sobre test set (muestra representativa)
metrics_test = wrapper_ll.evaluate(X_test[:500], y_test[:500])

In [ ]:
# Evaluación sobre validation set (muestra representativa)
metrics_val = wrapper_ll.evaluate(X_va[:500], y_va[:500])